<a href="https://colab.research.google.com/github/RCortez25/Scientific-Machine-Learning/blob/main/Differential_equations/Lotka_Volterra_UDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
---

In [ ]:
using JLD, Lux, DifferentialEquations, Optimization, OptimizationOptimJL
using Random, Plots, ComponentArrays, ModelingToolkit

random_number_generator = Random.default_rng()
Random.seed!(random_number_generator, 42)

#------ Generate ground-truth data
@parameters α β δ γ
@independent_variables t
@variables x(t) y(t)
Dt = Differential(t)

eqs = [
    Dt(x) ~ α*x - β*x*y,
    Dt(y) ~ -δ*y + γ*x*y
]

@named system = ODESystem(eqs, t, [x, y], [α, β, δ, γ])
simplified = structural_simplify(system)

N_days = 25

parameter_map = Dict(α => 1.0, β => 0.02, δ => 0.5, γ => 0.02)
initial_conditions = Dict(x => 20, y => 10)
timespan = (0.0, N_days)

The code is for generating ground truth data with the real Lotka-Volterra system of equations. Note that in this case que values of the parameters

`α => 1.0, β => 0.02, δ => 0.5, γ => 0.02`

were selected rather pedagogically, in order to have a "nice" system.

In [ ]:
problem = ODEProblem(
    simplified,
    merge(initial_conditions, parameter_map),
    timespan
)

solution = solve(problem; saveat = 0.1)
solution = Array(solution) # solution[1,:] = y, solution[2,:] = x

x_ground_truth = solution[2, :]
y_ground_truth = solution[1, :]

Here, the ODE problem is generated as `problem`. The problem is solved with default parameters (not a specific integrator).

The important thing to note, is that, for some reason, `structural_simplify` rearranges variables, so that instead of having `(x,y)` one has `(y,x)`. That's why, in order to retrieve the `x` values, we have to use the second index as in

`x_ground_truth = solution[2, :]`

and the first index for `y`

`y_ground_truth = solution[1, :]`